In [1]:
import ee
import geemap

# --- 步骤 1: 初始化并认证 Google Earth Engine ---
try:
    ee.Initialize(project='your-project')
except Exception as e:
    ee.Authenticate()
    ee.Initialize(project='your-project')

print("Google Earth Engine 初始化成功！")

# --- 步骤 2: 定义最终的研究区域 ---
# 直接使用您从NOAA网站上重新划定的、数据覆盖更好的新坐标
final_min_lon, final_max_lon = -82.57869, -82.56298
final_min_lat, final_max_lat = 27.65863, 27.66957

# 创建最终的研究区域几何体
final_aoi = ee.Geometry.BBox(final_min_lon, final_min_lat, final_max_lon, final_max_lat)
print("已定义最终的研究区域。")


# --- 步骤 3: 查找并裁剪最终的卫星图像 ---
sentinel_collection = ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED')
# 直接使用最终的AOI进行筛选
filtered_collection = sentinel_collection.filterBounds(final_aoi).filterDate('2025-01-01', '2025-03-31').filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 10))
image = filtered_collection.sort('CLOUDY_PIXEL_PERCENTAGE').first()
# 直接使用最终的AOI进行裁剪
final_image_to_export = image.clip(final_aoi)
image_id_info = image.id().getInfo()
print(f"找到的最清晰图像ID: {image_id_info}")


# --- 步骤 4: 启动导出任务 ---
# 我们只导出科学分析需要的波段
export_image = final_image_to_export.select(['B4', 'B3', 'B2'])

# 定义导出参数，使用新的描述和文件名
export_task = ee.batch.Export.image.toDrive(
    image=export_image,
    description='Final_Clean_Area_TampaBay',       # 新的导出任务描述
    folder='GEE_Exports',                          # 在您Google Drive里保存的文件夹名
    fileNamePrefix='tampa_bay_s2_final_subset',    # 新的文件名前缀
    region=final_aoi.getInfo()['coordinates'],     # 导出的区域
    scale=10,                                      # 导出的分辨率（10米）
    crs='EPSG:4326',                               # 坐标系 (WGS84)
    maxPixels=1e10,
    fileFormat='GeoTIFF'
)

# 启动任务
export_task.start()

print("\n导出任务已启动！")
print("请访问Google Earth Engine的任务管理器来监控进度:")
print("https://code.earthengine.google.com/tasks")
print("任务完成后，您就可以在您的Google Drive中找到最终的、与新水深数据完美对齐的图像文件了。")

C:\Users\pc\anaconda3\envs\earth_engine\lib\site-packages\google\api_core\_python_version_support.py:266: FutureWarning: You are using a Python version (3.10.0) which Google will stop supporting in new releases of google.api_core once it reaches its end of life (2026-10-04). Please upgrade to the latest Python version, or at least Python 3.11, to continue receiving updates for google.api_core past that date.
  warnings.warn(message, FutureWarning)


Google Earth Engine 初始化成功！
已定义最终的研究区域。
找到的最清晰图像ID: 20250112T160529_20250112T160531_T17RLL

导出任务已启动！
请访问Google Earth Engine的任务管理器来监控进度:
https://code.earthengine.google.com/tasks
任务完成后，您就可以在您的Google Drive中找到最终的、与新水深数据完美对齐的图像文件了。


In [5]:
import rasterio
import numpy as np
from PIL import Image

# --- 1. 定义文件路径 ---
# 请修改为您的GeoTIFF文件的路径
input_geotiff_path = './tampa_bay_s2_final_subset.tif' 

# 您希望保存的PNG文件的路径和名称
output_png_path = 'output_rgb_image_for_model.png'

print(f"准备处理文件: {input_geotiff_path}")

# --- 2. 使用 rasterio 读取 GeoTIFF 文件 ---
try:
    with rasterio.open(input_geotiff_path) as dataset:
        print("GeoTIFF 文件成功打开。")
        
        # 读取所有波段 (B4, B3, B2) 到一个NumPy数组中
        # 顺序是 红, 绿, 蓝
        # 数组的形状将是 (通道数, 高度, 宽度)，例如 (3, 891, 800)
        raw_data = dataset.read()
        print(f"读取的数据维度: {raw_data.shape}")
        print(f"原始数据类型: {raw_data.dtype}")

        # --- 3. 这是最关键的一步：将科学数值“拉伸”到0-255的视觉范围 ---
        
        # 根据我们之前在GEE中反复试验得到的最佳可视化参数
        # 这些值决定了图像的“亮度和对比度”
        min_val = 0
        max_val = 1500 # 这是我们最终确定的、效果最好的'max'值
        
        print(f"正在将数值从 [{min_val}, {max_val}] 范围拉伸到 [0, 255]...")
        
        # 首先，使用np.clip将所有值限制在min_val和max_val之间
        # 这可以防止极端值（如云的亮点）影响整体图像的亮度
        clipped_data = np.clip(raw_data, min_val, max_val)
        
        # 其次，进行线性拉伸
        # (像素值 - 最小值) / (最大值 - 最小值) -> 得到0到1之间的浮点数
        # 然后乘以255 -> 得到0到255之间的浮点数
        scaled_data = ((clipped_data - min_val) / (max_val - min_val)) * 255
        
        # 最后，将数据类型转换为8位无符号整数 (uint8)
        # 这是图像文件所要求的标准格式
        rgb_data_uint8 = scaled_data.astype(np.uint8)
        print(f"数据已成功转换为 uint8 类型。")

        # --- 4. 调整数组维度以适应图像库的要求 ---
        
        # rasterio读取的顺序是 (通道, 高, 宽)
        # Pillow库需要的顺序是 (高, 宽, 通道)
        # 我们需要用np.transpose来交换轴的顺序
        rgb_data_hwc = np.transpose(rgb_data_uint8, (1, 2, 0))
        print(f"数组维度已调整为: {rgb_data_hwc.shape}")

        # --- 5. 使用Pillow创建并保存最终的RGB图像 ---
        
        # 从NumPy数组创建Pillow图像对象
        image = Image.fromarray(rgb_data_hwc)
        
        # 保存为PNG文件
        image.save(output_png_path)
        
        print("-" * 50)
        print("成功！")
        print(f"模型可用的RGB图像已保存到: {output_png_path}")
        print("-" * 50)

except FileNotFoundError:
    print(f"错误：找不到GeoTIFF文件。请检查路径是否正确: {input_geotiff_path}")
except Exception as e:
    print(f"处理过程中发生错误: {e}")

准备处理文件: ./tampa_bay_s2_final_subset.tif
GeoTIFF 文件成功打开。
读取的数据维度: (3, 231, 337)
原始数据类型: uint16
正在将数值从 [0, 1500] 范围拉伸到 [0, 255]...
数据已成功转换为 uint8 类型。
数组维度已调整为: (231, 337, 3)
--------------------------------------------------
成功！
模型可用的RGB图像已保存到: output_rgb_image_for_model.png
--------------------------------------------------
